In [1]:
import pandas as pd
input_file = pd.read_excel('Downloads/Copy of Input.xlsx')
output_f = pd.read_excel('Downloads/Copy of Output Data Structure.xlsx')
output_file = output_f.copy()
input_file.head()

,URL_ID,URL
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...
1,38.0,https://insights.blackcoffer.com/what-if-the-c...
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...
3,40.0,https://insights.blackcoffer.com/will-machine-...
4,41.0,https://insights.blackcoffer.com/will-ai-repla...


In [2]:
output_file.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,40.0,https://insights.blackcoffer.com/will-machine-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,41.0,https://insights.blackcoffer.com/will-ai-repla...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

path = "C:\Program Files (x86)\chromedriver.exe"

service = Service(executable_path=path)
driver = webdriver.Chrome(service=service)

In [4]:
def extract(url):
    driver.get(url)
    title = driver.find_element(By.XPATH, '/html/body/div[6]/article/div[1]/div[1]/div[2]/div/header/h1').text
    article = driver.find_element(By.XPATH, '/html/body/div[6]/article/div[2]/div/div[1]/div/div[1]').text
    return str(title)+' '+str(article)

### Text cleaning:

In [5]:
import string
def text_cleaning(text):
    #Lowering the text
    text = text.lower()
    
    #removing the new line escape character
    text = text.replace('\n',' ')
    
    #getting punctuations which are not in string module
    letters = sorted(set(text))
    l = (string.ascii_lowercase + string.digits+'.'+' ')
    l = [i for i in l]
    l =[i for i in letters if i not in l]
    
    #Having a list of sentences
    sentences = text.split('.')
    
    #Removing punctuations 
    s_l = len(sentences)
    p = ''.join(set(string.punctuation + ''.join(l)))
    for i in range(s_l):
        for j in p:
            sentences[i] = sentences[i].replace(j,'')
            
    # converting each sentence into a list of words
    for i in range(s_l):
        sentences[i] = sentences[i].split(' ')
    
    #combining lists of words into one list
    words = []
    for i in range(s_l):
        words.extend(sentences[i])
       
    # removing empty items
    w_i = 0
    len_w = len(words)
    while w_i < len_w:
        if words[w_i] == '' or words[w_i] ==' ':
            del words[w_i]
            w_i-=1
        len_w = len(words)
        w_i+=1
    
    # Removing stop words
    stopwords = nltk.corpus.stopwords.words('english')
    words = [j for j in words if j not in stopwords]
    
    #returning the cleaned text
    return sentences, words
    

Now since the cleaning is done I'll find values of each column for the output file

### To find:
* Positive score
* Negative score
* Polarity score
* Subjectivivty score
* Average sentence length
* Percentage of complex words
* FOG index
* Average number of words per sentence
* Complex word count
* Word count
* Syllable per word
* Personal Pronouns
* Average Word Length

#### Finding Positive score, Negative score, Polarity score, and Subjectivity score

In [6]:
import nltk
def all_scores(words):    
    w_l = len(words)

    #Positive score and negative score
    num_pos = nltk.corpus.opinion_lexicon.positive()
    num_neg = nltk.corpus.opinion_lexicon.negative()
    positive_score = 0
    negative_score = 0
    for i in range(w_l):
        if words[i] in num_pos:
            positive_score+=1
        elif words[i] in num_neg:
            negative_score+=1

    #Polarity score
    polarity_score = (positive_score - negative_score)/((positive_score + negative_score)+0.000001)

    #Subjectivity score
    subjectivity_score = (positive_score+negative_score)/(len(words)+0.000001)
    
    #returning the scores
    return list([positive_score, negative_score, polarity_score, subjectivity_score])


In [7]:
#### Finding average sentence lenth, percentage of complex words, and fog index

In [8]:
import requests
def readability_analysis(sentences, words):
    #formula for average sentence length
    avg_sentence_len = len(list(set(words)))/len(sentences)
    
    #finding percentage of complex numbers
    #geting all the dificult words for applying dale chal formula
    file = 'https://gist.githubusercontent.com/Abhishek-P/e00edcc6f508640fe24f263f5836a7dc/raw/166225e09fb8b554deff37ec344ad5ca40dab2fb/dale-chall-3000-words.txt'
    dcf = requests.get(file).text.replace('\t','').split(' ')
    no_complex = 0
    for i in words:
        if i in dcf:
            no_complex+=1
    pct_complex_wds = no_complex/len(words)
    
    fog_index = 0.4*(avg_sentence_len+pct_complex_wds)
    
    return list([avg_sentence_len, pct_complex_wds, fog_index])
    

In [9]:
def syllable_count(word):
    word = word.lower()
    count = 0
    vowels = "aeiouy"
    
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e") or word.endswith("ed") or word.endswith("es"):
        count -= 1    
    return count

In [10]:
output_file.columns

Index(['URL_ID', 'URL', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE',
       'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH',
       'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX',
       'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT',
       'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH'],
      dtype='object')

In [11]:
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize

def pronoun_count(text):
    pos = pos_tag(word_tokenize(text))

    pos[:5]
    prp = []
    for i in pos:
        if 'PRP' in i:
            prp.append(i[0])
    prp_count = len(prp)
    return prp_count

In [12]:
columns = list(output_file.columns)
print(columns)

['URL_ID', 'URL', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE', 'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX', 'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT', 'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH']


In [30]:
def update(v,i):
    columns = list(output_file.columns)
    for j in range(2,len(columns)):
        output_file[columns[j]].iloc[i] = v[j]

In [32]:
import numpy as np
import time
start = time.time()
for i in range(5): #len(output_file)
    url = output_file['URL'].iloc[i]
    
    #extracting the 
    text = extract(url)
    
    sentences, words = text_cleaning(text)
    #getting score in the form of list [positive_score, negative_score, polarity_score, subjectivity_score] 
    scores = all_scores(words)
    
    #calling readability analysis function for [avg_sentence_len, pct_complex_wds, fog_index]
    ra  = readability_analysis(sentences,words)
    
    # average number of words per sentce
    avg_wds_sentence = len(words)/len(text)
    
    #complex word count and average syllable per word
    complex_words = []
    slb_word = []
    check = 0
    for x in words:
        check+=1
        c = syllable_count(x)
        slb_word.append(c)
        if c > 2:
            complex_words.append(x)
    complex_wd_cnt = len(complex_words)
    #lst = lambda x: f'{len([i for i in slb_word if int(i) == x])} Words: {x} syllables'
    #syllable_per_word =str([lst(i) for i in set(slb_word)])
    syllable_per_word = round(np.median(slb_word))
    
    #word count
    word_count = len(words)
    
    #Personal pronoun count
    prp_count = pronoun_count(text)
    
    #Average word length
    len_char = 0
    for k in words:
        len_char = len_char + len(k)
    avg_word_length = len_char/len(words) 
    
    value_list = []
    value_list.extend(scores)
    value_list.extend(ra)
    value_list.extend([avg_wds_sentence,complex_wd_cnt,word_count,syllable_per_word,prp_count,avg_word_length])
    
    values = dict(enumerate(value_list, 2))
    update(values,i)
    
    
stop = time.time()
print(stop-start/60,'minutes')
    

C:\Users\Shah\AppData\Local\Temp\ipykernel_6944\2817793839.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_file[columns[j]].iloc[i] = v[j]


216.33589696884155 s


In [33]:
output_file.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,79.0,34.0,0.398230,0.098261,9.410256,0.179130,3.835755,0.094093,401.0,1150.0,2.0,22.0,7.217391
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,71.0,37.0,0.314815,0.141361,6.760563,0.349476,2.844016,0.089566,197.0,764.0,2.0,52.0,6.502618
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,74.0,36.0,0.345455,0.109235,7.376471,0.262165,3.055454,0.090120,354.0,1007.0,2.0,28.0,7.241311
3,40.0,https://insights.blackcoffer.com/will-machine-...,86.0,28.0,0.508772,0.124863,6.325301,0.349398,2.669880,0.091823,231.0,913.0,2.0,49.0,6.383352
4,41.0,https://insights.blackcoffer.com/will-ai-repla...,69.0,28.0,0.422680,0.096326,8.170732,0.308838,3.391828,0.092182,287.0,1007.0,2.0,53.0,6.734856


In [34]:
output_file.to_excel('updated_op_file.xlsx')